In [ ]:
import pandas as pd
import math

data = {
    'Outlook': ['Sunny', 'Sunny', 'Overcast', 'Rainy', 'Rainy', 'Rainy', 'Overcast',
                'Sunny', 'Sunny', 'Rainy', 'Sunny', 'Overcast', 'Overcast', 'Rainy'],
    'Temperature': ['Hot', 'Hot', 'Hot', 'Mild', 'Cool', 'Cool', 'Cool',
                   'Mild', 'Cool', 'Mild', 'Mild', 'Mild', 'Hot', 'Mild'],
    'Humidity': ['High', 'High', 'High', 'High', 'Normal', 'Normal', 'Normal',
                'High', 'Normal', 'Normal', 'Normal', 'High', 'Normal', 'High'],
    'Windy': ['False', 'True', 'False', 'False', 'False', 'True', 'True',
             'False', 'False', 'False', 'True', 'True', 'False', 'True'],
    'Play': ['No', 'No', 'Yes', 'Yes', 'Yes', 'No', 'Yes',
            'No', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'No']
}

df = pd.DataFrame(data)

def entropy(target_col):
    elements, counts = pd.unique(target_col), target_col.value_counts()
    entropy_val = 0
    for i in range(len(elements)):
        prob = counts[elements[i]] / len(target_col)
        entropy_val -= prob * math.log2(prob)
    return entropy_val

# Calculate information gain
def info_gain(data, split_attribute, target_name="Play"):
    total_entropy = entropy(data[target_name])
    vals, counts = pd.unique(data[split_attribute]), data[split_attribute].value_counts()

    weighted_entropy = 0
    for i in range(len(vals)):
        subset = data[data[split_attribute] == vals[i]][target_name]
        weighted_entropy += (counts[vals[i]] / len(data)) * entropy(subset)

    return total_entropy - weighted_entropy


def id3(data, original_data, features, target_attribute="Play", parent_node_class=None):
    if len(pd.unique(data[target_attribute])) == 1:
        return pd.unique(data[target_attribute])[0]

    if len(data) == 0:
        return pd.unique(original_data[target_attribute])[
            pd.value_counts(original_data[target_attribute]).argmax()]

    if len(features) == 0:
        return parent_node_class

    parent_node_class = pd.unique(data[target_attribute])[
        pd.value_counts(data[target_attribute]).argmax()]

    info_gains = {feature: info_gain(data, feature, target_attribute)
                 for feature in features}

    best_feature = max(info_gains.items(), key=lambda x: x[1])[0]


    tree = {best_feature: {}}
    features = [f for f in features if f != best_feature]

    # Grow tree for each value of best feature
    for value in pd.unique(data[best_feature]):
        sub_data = data[data[best_feature] == value]
        subtree = id3(sub_data, original_data, features, target_attribute,
                     parent_node_class)
        tree[best_feature][value] = subtree

    return tree

features = list(df.columns[:-1])  # All columns except 'Play'
decision_tree = id3(df, df, features)

# Pretty print the tree
def print_tree(tree, level=0):
    if not isinstance(tree, dict):
        print("\t" * level + f"Leaf: {tree}")
        return
    for attribute, branches in tree.items():
        print("\t" * level + f"{attribute}")
        for value, subtree in branches.items():
            print("\t" * (level + 1) + f"{value} ->")
            print_tree(subtree, level + 2)

print("Decision Tree:")
print_tree(decision_tree)



Decision Tree:
Outlook
	Sunny ->
		Humidity
			High ->
				Leaf: No
			Normal ->
				Leaf: Yes
	Overcast ->
		Leaf: Yes
	Rainy ->
		Windy
			False ->
				Leaf: Yes
			True ->
				Leaf: No


<ipython-input-1-b268014b635c>:52: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  pd.value_counts(data[target_attribute]).argmax()]
<ipython-input-1-b268014b635c>:52: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  pd.value_counts(data[target_attribute]).argmax()]
<ipython-input-1-b268014b635c>:52: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  pd.value_counts(data[target_attribute]).argmax()]
